In [1]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.prompts import PromptTemplate

from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

import os, requests, time, sys 

from pathlib import Path

sys.path.append(os.path.abspath(os.path.join('..')))

# from mongo_utils import *

from dotenv import load_dotenv

from tqdm import tqdm

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
load_dotenv(dotenv_path="../.env.local")

True

In [3]:
model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), 
        model_name = 'gpt-4',
        temperature=0.3)


In [5]:
loader = WebBaseLoader(
    web_paths=("https://www.nspe.org/resources/ethics/code-ethics",
               "https://ethics.acm.org/code-of-ethics/software-engineering-code/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

ValueError: Expected IDs to be a non-empty list, got 0 IDs